#ACTIVIDAD 3 ML II

En esta actividad se aborda el problema de predicción de fuga de clientes (churn) utilizando enfoques de clasificación distintos a los vistos anteriormente. Mientras que en actividades previas se trabajó con regresión logística y modelos basados en árboles, en esta oportunidad el foco está en Naïve Bayes y Support Vector Machines (SVM), dos familias de modelos con fundamentos conceptuales diferentes.

El objetivo principal es analizar cómo estos modelos se comportan frente a un problema real de clasificación binaria con desbalance de clases, evaluando no solo su desempeño predictivo, sino también aspectos como el costo computacional, la estabilidad de los resultados y la facilidad de interpretación. Para asegurar una comparación justa, se utiliza el mismo dataset y el mismo flujo de preprocesamiento definidos en las actividades anteriores.

Naïve Bayes se presenta como un modelo probabilístico simple y eficiente, basado en el supuesto de independencia condicional entre las variables explicativas. A pesar de que este supuesto rara vez se cumple estrictamente en datos reales, el modelo suele ofrecer resultados competitivos como línea base, especialmente por su rapidez de entrenamiento y evaluación.

Por otro lado, Support Vector Machines permiten construir fronteras de decisión más flexibles. En esta actividad se analizan dos variantes: una versión lineal, que busca una separación simple entre clases, y una versión con kernel RBF, capaz de capturar relaciones no lineales más complejas. En ambos casos, se estudia el impacto del escalamiento de variables y de la selección de hiperparámetros en el desempeño del modelo.

La evaluación se realiza utilizando métricas adecuadas para problemas desbalanceados, como F1 y PR-AUC, además de curvas ROC y Precision–Recall. Finalmente, se discuten los resultados desde una perspectiva tanto técnica como de negocio, considerando el contexto de campañas de retención de clientes, donde identificar correctamente a quienes tienen mayor probabilidad de abandono resulta especialmente relevante.

In [ ]:
# Librerías base
import numpy as np
import pandas as pd
import time
from scipy.stats import loguniform

# Visualización
import matplotlib.pyplot as plt

# Sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, Binarizer
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# Métricas y curvas
from sklearn.metrics import (
    roc_curve, precision_recall_curve
)

from utils import *

In [ ]:
df = pd.read_csv("data-churn.csv")

df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

target = "Churn"

X = df.drop(columns=[target]).copy()
y = df[target].copy()

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Shape:", df.shape)
print("Target distribution:\n", y.value_counts(normalize=True))



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



In [ ]:
ohe_dense = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

numeric_transformer_nb = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer_nb = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe_dense)
])

preprocessor_nb = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_nb, numeric_features),
        ("cat", categorical_transformer_nb, categorical_features),
    ],
    remainder="drop"
)

pipe_nb = Pipeline(steps=[
    ("preprocessor", preprocessor_nb),
    ("clf", GaussianNB())
])



In [ ]:
param_grid_nb = [
    {
        "clf": [GaussianNB()],
        "clf__var_smoothing": np.logspace(-12, -7, 6)
    },
    {
        "clf": [BernoulliNB()],
        "clf__alpha": [0.1, 0.5, 1.0, 2.0, 5.0],
        "clf__binarize": [0.0, 0.5, 1.0]
    }
]

scoring_main = "f1"  # o "average_precision"

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

t0 = time.perf_counter()
grid_nb = GridSearchCV(
    estimator=pipe_nb,
    param_grid=param_grid_nb,
    scoring=scoring_main,
    cv=cv,
    n_jobs=-1,
    verbose=0
)
grid_nb.fit(X_train, y_train)
t_nb = time.perf_counter() - t0



In [ ]:
print("=== Naïve Bayes ===")
print("Best params:", grid_nb.best_params_)
print(f"CV best {scoring_main}: {grid_nb.best_score_:.4f}")
print(f"GridSearch time (s): {t_nb:.2f}")

best_nb = grid_nb.best_estimator_



In [ ]:
nb_metrics, nb_pred, nb_score = report_test_metrics(best_nb, X_test, y_test, label="NaiveBayes")
print(pd.Series(nb_metrics))

plot_confusion(y_test, nb_pred, title="Naïve Bayes — Confusion Matrix (test)")

fpr, tpr, _ = roc_curve(y_test, nb_score)
prec, rec, _ = precision_recall_curve(y_test, nb_score)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], linestyle="--")
ax.set_title("Naïve Bayes — ROC (test)")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
plt.show()

fig, ax = plt.subplots()
ax.plot(rec, prec)
ax.set_title("Naïve Bayes — Precision–Recall (test)")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
plt.show()

In [ ]:
ohe_sparse = OneHotEncoder(handle_unknown="ignore")

numeric_transformer_svm = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer_svm = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe_sparse)
])

preprocessor_svm = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_svm, numeric_features),
        ("cat", categorical_transformer_svm, categorical_features),
    ],
    remainder="drop"
)


In [ ]:
pipe_linsvm = Pipeline(steps=[
    ("preprocessor", preprocessor_svm),
    ("clf", CalibratedClassifierCV(
        estimator=LinearSVC(dual=True, max_iter=20000),
        method="sigmoid",
        cv=3
    ))
])

param_grid_linsvm = {
    "clf__estimator__C": [0.01, 0.1, 1.0, 10.0, 100.0]
}

pipe_rbf = Pipeline(steps=[
    ("preprocessor", preprocessor_svm),
    ("clf", SVC(kernel="rbf", probability=False))
])

param_grid_rbf = {
    "clf__C": [0.1, 1.0, 10.0, 100.0],
    "clf__gamma": ["scale", "auto", 1e-3, 1e-2, 1e-1]
}

scoring_main = "f1"  # o "average_precision"


In [ ]:

print("\n=== SVM Lineal (Grid) ===")
grid_linsvm = GridSearchCV(pipe_linsvm, param_grid_linsvm, scoring=scoring_main, cv=cv, n_jobs=-1)
grid_linsvm, t_grid_lin = timed_search(grid_linsvm, X_train, y_train, label="Grid LinearSVM")

print("\n=== SVM RBF (Grid) ===")
grid_rbf = GridSearchCV(pipe_rbf, param_grid_rbf, scoring=scoring_main, cv=cv, n_jobs=-1)
grid_rbf, t_grid_rbf = timed_search(grid_rbf, X_train, y_train, label="Grid RBF-SVM")

print("\n=== SVM Lineal (Random) ===")
rand_linsvm = RandomizedSearchCV(
    pipe_linsvm,
    param_distributions={"clf__estimator__C": loguniform(1e-3, 1e2)},
    n_iter=20,
    scoring=scoring_main,
    cv=cv,
    n_jobs=-1,
    random_state=42
)
rand_linsvm, t_rand_lin = timed_search(rand_linsvm, X_train, y_train, label="Random LinearSVM")

print("\n=== SVM RBF (Random) ===")
rand_rbf = RandomizedSearchCV(
    pipe_rbf,
    param_distributions={"clf__C": loguniform(1e-2, 1e3), "clf__gamma": loguniform(1e-4, 1e0)},
    n_iter=30,
    scoring=scoring_main,
    cv=cv,
    n_jobs=-1,
    random_state=42
)
rand_rbf, t_rand_rbf = timed_search(rand_rbf, X_train, y_train, label="Random RBF-SVM")

# Elegimos el mejor (según score CV) para cada familia
best_lin = grid_linsvm.best_estimator_ if grid_linsvm.best_score_ >= rand_linsvm.best_score_ else rand_linsvm.best_estimator_
best_rbf = grid_rbf.best_estimator_ if grid_rbf.best_score_ >= rand_rbf.best_score_ else rand_rbf.best_estimator_

# ---- Reporte final en test (métricas, curvas) :contentReference[oaicite:12]{index=12}
lin_metrics, lin_pred, lin_score = report_test_metrics(best_lin, X_test, y_test, label="SVM Linear (best)")
rbf_metrics, rbf_pred, rbf_score = report_test_metrics(best_rbf, X_test, y_test, label="SVM RBF (best)")

results = pd.DataFrame([nb_metrics, lin_metrics, rbf_metrics]).set_index("model")


In [ ]:
print("\n=== Test metrics summary ===")
display(results)

plot_confusion(y_test, lin_pred, title="SVM Linear — Confusion Matrix (test)")
plot_confusion(y_test, rbf_pred, title="SVM RBF — Confusion Matrix (test)")

# Curvas ROC/PR en test


plot_roc_pr(y_test, lin_score, title="SVM Linear (best)")
plot_roc_pr(y_test, rbf_score, title="SVM RBF (best)")

# ============================================================
# 2e) Desbalance: repetir mejor SVM con class_weight='balanced' :contentReference[oaicite:13]{index=13}
# ============================================================

# Reentrena SOLO la mejor familia (ejemplo: RBF). Puedes hacerlo para ambos si quieres.
best_rbf_balanced = Pipeline(steps=[
    ("preprocessor", preprocessor_svm),
    ("clf", SVC(kernel="rbf", class_weight="balanced"))
])


In [ ]:
# Usamos los mejores hiperparámetros encontrados
best_params = {}
if hasattr(best_rbf, "get_params"):
    bp = best_rbf.get_params()
    # Extraemos C y gamma si venían
    if "clf__C" in bp: best_params["clf__C"] = bp["clf__C"]
    if "clf__gamma" in bp: best_params["clf__gamma"] = bp["clf__gamma"]

best_rbf_balanced.set_params(**best_params)
best_rbf_balanced.fit(X_train, y_train)

rbf_bal_metrics, rbf_bal_pred, rbf_bal_score = report_test_metrics(
    best_rbf_balanced, X_test, y_test, label="SVM RBF balanced"
)

print("\n=== SVM RBF balanced (test) ===")
print(pd.Series(rbf_bal_metrics))
plot_confusion(y_test, rbf_bal_pred, title="SVM RBF balanced — Confusion Matrix (test)")
plot_roc_pr(y_test, rbf_bal_score, title="SVM RBF balanced")

# Actividad 3 – Machine Learning II  
## Naïve Bayes y SVM para predicción de Churn

Este notebook se basa en el código base entregado. Las celdas originales se mantienen y se agregan secciones nuevas para completar los requerimientos de la Actividad 3.

A lo largo del desarrollo se utiliza el mismo dataset y el mismo preprocesamiento base definido en actividades previas (imputación, one-hot encoding y escalamiento de variables numéricas), con el objetivo de mantener comparabilidad entre modelos.


In [ ]:
# ==========================================================
# Ajuste de compatibilidad: si no existe utils.py en el entorno
# ==========================================================
# El código base importa "from utils import *". Si ese archivo no está disponible
# en tu entorno (por ejemplo, al ejecutar en otra máquina), definimos aquí las
# funciones mínimas necesarias para continuar sin errores.
#
# Nota: si utils.py sí existe, esta celda no cambia nada relevante.
try:
    _ = evaluate_model_cv_detailed  # noqa: F821
    _ = get_feature_names_from_preprocessor  # noqa: F821
    print("✅ utils disponibles: se usará la implementación original.")
except Exception:
    print("⚠️ utils no detectado: se crearán funciones auxiliares mínimas.")
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.metrics import (
        confusion_matrix, classification_report,
        accuracy_score, precision_score, recall_score, f1_score,
        roc_curve, auc, precision_recall_curve, average_precision_score
    )

    def get_feature_names_from_preprocessor(preprocessor):
        feature_names = []
        for name, transformer, cols in preprocessor.transformers_:
            if name == "remainder" and transformer == "drop":
                continue
            if hasattr(transformer, "named_steps"):
                last = list(transformer.named_steps.values())[-1]
                if hasattr(last, "get_feature_names_out"):
                    try:
                        names = last.get_feature_names_out(cols)
                    except Exception:
                        names = last.get_feature_names_out()
                    feature_names.extend(names)
                else:
                    feature_names.extend(cols)
            else:
                if hasattr(transformer, "get_feature_names_out"):
                    try:
                        names = transformer.get_feature_names_out(cols)
                    except Exception:
                        names = transformer.get_feature_names_out()
                    feature_names.extend(names)
                else:
                    feature_names.extend(cols)
        return [str(n) for n in feature_names]

    def evaluate_model_cv_detailed(model, X, y, cv, plot_curves=True, verbose=True):
        per_fold = []
        roc_curves, pr_curves = [], []
        mean_fpr = np.linspace(0, 1, 200)
        mean_recall = np.linspace(0, 1, 200)

        y_true_all, y_pred_all, y_proba_all = [], [], []

        for fold, (tr, te) in enumerate(cv.split(X, y), start=1):
            X_tr, X_te = X.iloc[tr], X.iloc[te]
            y_tr, y_te = y.iloc[tr], y.iloc[te]

            model.fit(X_tr, y_tr)
            # Probabilidades: si no existe predict_proba, usamos decision_function reescalada
            if hasattr(model, "predict_proba"):
                y_proba = model.predict_proba(X_te)[:, 1]
            else:
                scores = model.decision_function(X_te)
                # Reescalado simple a (0,1) para poder graficar y calcular PR-AUC
                y_proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)

            y_pred = (y_proba >= 0.5).astype(int)

            acc = accuracy_score(y_te, y_pred)
            prec = precision_score(y_te, y_pred, zero_division=0)
            rec = recall_score(y_te, y_pred, zero_division=0)
            f1 = f1_score(y_te, y_pred, zero_division=0)
            pr_auc = average_precision_score(y_te, y_proba)

            fpr, tpr, _ = roc_curve(y_te, y_proba)
            roc_auc = auc(fpr, tpr)

            per_fold.append({
                "fold": fold, "accuracy": acc, "precision": prec, "recall": rec,
                "f1": f1, "roc_auc": roc_auc, "pr_auc": pr_auc
            })

            tpr_i = np.interp(mean_fpr, fpr, tpr)
            tpr_i[0] = 0.0
            roc_curves.append(tpr_i)

            p, r, _ = precision_recall_curve(y_te, y_proba)
            order = np.argsort(r)
            r_s, p_s = r[order], p[order]
            p_i = np.interp(mean_recall, r_s, p_s)
            pr_curves.append(p_i)

            y_true_all.extend(y_te)
            y_pred_all.extend(y_pred)
            y_proba_all.extend(y_proba)

        df_fold = pd.DataFrame(per_fold)
        summary = pd.DataFrame({
            "mean": df_fold.drop(columns=["fold"]).mean(),
            "std": df_fold.drop(columns=["fold"]).std()
        })

        if verbose:
            print("Resultados por fold:")
            display(df_fold)
            print("\nResumen (promedio y desviación estándar):")
            display(summary)

            cm = confusion_matrix(y_true_all, y_pred_all)
            print("\nMatriz de confusión (global):\n", cm)
            print("\nReporte de clasificación (global):\n", classification_report(y_true_all, y_pred_all, digits=3))

        curves = None
        if plot_curves:
            mean_tpr = np.mean(roc_curves, axis=0)
            mean_tpr[-1] = 1.0
            mean_roc_auc = auc(mean_fpr, mean_tpr)

            mean_precision = np.mean(pr_curves, axis=0)
            mean_pr_auc = auc(mean_recall, mean_precision)

            plt.figure()
            plt.plot(mean_fpr, mean_tpr, label=f"ROC promedio (AUC≈{mean_roc_auc:.3f})")
            plt.plot([0, 1], [0, 1], linestyle="--")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate (Recall)")
            plt.title("Curva ROC promedio")
            plt.legend()
            plt.show()

            plt.figure()
            plt.plot(mean_recall, mean_precision, label=f"PR promedio (AUC≈{mean_pr_auc:.3f})")
            plt.xlabel("Recall")
            plt.ylabel("Precision")
            plt.title("Curva Precision–Recall promedio")
            plt.legend()
            plt.show()

            curves = {"mean_fpr": mean_fpr, "mean_tpr": mean_tpr,
                      "mean_recall": mean_recall, "mean_precision": mean_precision}

        return df_fold, summary, curves


## Primer Paso. Naïve Bayes

En esta sección se implementa un clasificador **Naïve Bayes**. La idea principal es estimar la probabilidad de churn a partir de una combinación “simple” de evidencias (las variables), asumiendo que, dado el estado de churn, las variables se comportan como si fueran independientes entre sí.

En la práctica, esta independencia no siempre se cumple. Aun así, Naïve Bayes suele funcionar bien como modelo base rápido, y su costo computacional es bajo.


In [ ]:
# =========================
# 1.1 Variante de Naïve Bayes
# =========================
# Dado que el preprocesamiento incluye variables continuas escaladas y variables one-hot,
# trabajaremos con GaussianNB, que modela features continuas con distribuciones normales.
# Como GaussianNB espera una matriz densa, convertimos la salida del preprocesador a densa.

from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

# Convertir a denso (si ya es denso, no cambia)
to_dense = FunctionTransformer(lambda x: x.toarray() if hasattr(x, "toarray") else x, accept_sparse=True)

pipe_nb = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("to_dense", to_dense),
    ("clf", GaussianNB())
])

# Hiperparámetro principal en GaussianNB: var_smoothing
# Probamos valores en escala logarítmica para cubrir rangos típicos sin exagerar el costo.
param_grid_nb = {
    "clf__var_smoothing": np.logspace(-12, -7, 6)
}

# Separamos un conjunto de test para reportar resultados finales (como pide la pauta)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

grid_nb = GridSearchCV(
    estimator=pipe_nb,
    param_grid=param_grid_nb,
    cv=cv5,
    scoring="average_precision",   # PR-AUC como métrica principal (clase churn suele ser minoritaria)
    n_jobs=-1,
    refit=True
)

t0 = time.perf_counter()
grid_nb.fit(X_train, y_train)
nb_search_time = time.perf_counter() - t0

print("Tiempo GridSearch (Naïve Bayes):", round(nb_search_time, 3), "s")
print("Mejores hiperparámetros (NB):")
pprint(grid_nb.best_params_)
print("Mejor score CV (PR-AUC):", grid_nb.best_score_)

best_nb = grid_nb.best_estimator_


### 1.2 Justificación (en palabras simples)

Se utiliza GaussianNB porque el conjunto final incluye variables numéricas continuas (escaladas) y variables categóricas codificadas. Este modelo asume que, dentro de cada clase (churn/no churn), las variables se distribuyen de forma aproximadamente “normal”. Aunque esta suposición es una simplificación, suele entregar resultados razonables como línea base y permite entrenar y evaluar muy rápido.

Además, se mantiene el mismo preprocesamiento que en actividades anteriores para que la comparación sea justa.


In [ ]:
# =========================
# 1.3 Evaluación final en TEST (métricas + curvas)
# =========================
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix,
    roc_curve, precision_recall_curve, auc
)

# Predicciones en test
best_nb.fit(X_train, y_train)
y_proba_nb = best_nb.predict_proba(X_test)[:, 1]
y_pred_nb = (y_proba_nb >= 0.5).astype(int)

print("Matriz de confusión (TEST):")
print(confusion_matrix(y_test, y_pred_nb))

print("\nMétricas (TEST):")
print("Accuracy :", round(accuracy_score(y_test, y_pred_nb), 3))
print("Precision:", round(precision_score(y_test, y_pred_nb, zero_division=0), 3))
print("Recall   :", round(recall_score(y_test, y_pred_nb, zero_division=0), 3))
print("F1       :", round(f1_score(y_test, y_pred_nb, zero_division=0), 3))
print("AUC-ROC  :", round(roc_auc_score(y_test, y_proba_nb), 3))
print("PR-AUC   :", round(average_precision_score(y_test, y_proba_nb), 3))

# Curvas en test
fpr, tpr, _ = roc_curve(y_test, y_proba_nb)
roc_auc = auc(fpr, tpr)

prec, rec, _ = precision_recall_curve(y_test, y_proba_nb)
pr_auc = average_precision_score(y_test, y_proba_nb)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.3f})")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("Naïve Bayes – Curva ROC (TEST)")
plt.legend()
plt.show()

plt.figure()
plt.plot(rec, prec, label=f"PR (AP={pr_auc:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Naïve Bayes – Curva Precision–Recall (TEST)")
plt.legend()
plt.show()


### 1.4 Evaluación con k-fold (curvas promedio)

Además del test final, se evalúa el modelo usando k-fold estratificado. Esto permite ver si el desempeño es consistente y no depende demasiado de una sola partición de datos.


In [ ]:
df_nb_cv, summary_nb_cv, curves_nb_cv = evaluate_model_cv_detailed(
    best_nb, X, y, cv=cv5, plot_curves=True, verbose=True
)


### 1.5 Dependencia entre predictores y su impacto en Naïve Bayes

Naïve Bayes supone independencia condicional entre variables. Para tener una idea de qué tan razonable es esa suposición, cuantificamos dependencia entre algunas variables numéricas mediante correlaciones.

Si dos variables están muy correlacionadas, Naïve Bayes puede “contar la misma evidencia dos veces”, lo que a veces empeora la calibración o el desempeño. Una forma común de mitigar esto es reducir dimensionalidad (por ejemplo, PCA), seleccionar variables o agrupar variables altamente redundantes.


In [ ]:
# Correlación entre variables numéricas (subconjunto)
num_cols = [c for c in ["tenure", "MonthlyCharges", "TotalCharges"] if c in df.columns]

# Asegurar que TotalCharges sea numérico si existe como texto
if "TotalCharges" in num_cols:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

corr = df[num_cols].corr()

plt.figure(figsize=(5, 4))
plt.imshow(corr, interpolation="nearest")
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
plt.yticks(range(len(num_cols)), num_cols)
plt.colorbar()
plt.title("Correlación entre variables numéricas (subconjunto)")
plt.tight_layout()
plt.show()

display(corr)


### 1.6 Resumen (Naïve Bayes)

En términos generales, Naïve Bayes destaca por su **rapidez** y por ser un buen punto de comparación inicial. Su principal limitación es que simplifica la relación entre variables, por lo que puede perder rendimiento cuando existen dependencias fuertes o relaciones no lineales relevantes.


## Segundo paso. Support Vector Machines (SVM)

En esta sección se entrenan y comparan dos variantes:
- **SVM lineal:** útil cuando buscamos una separación relativamente simple y valoramos interpretabilidad a nivel de pesos (en sentido conceptual).
- **SVM con kernel RBF:** útil cuando la relación entre variables y churn es más compleja y no se separa bien con una frontera lineal.

En SVM, el **escalamiento** es crítico porque el modelo se basa en distancias. Si una variable tiene valores muy grandes, puede dominar la decisión y distorsionar el resultado. Por eso mantenemos el mismo preprocesamiento con StandardScaler.


In [ ]:
from sklearn.svm import SVC

# Pipelines SVM (probability=True permite predict_proba; esto tiene costo computacional adicional)
pipe_svm_linear = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", SVC(kernel="linear", probability=True, random_state=RANDOM_STATE))
])

pipe_svm_rbf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", SVC(kernel="rbf", probability=True, random_state=RANDOM_STATE))
])


### 2.1 Selección de hiperparámetros (Grid Search vs Random Search)

Se comparan **GridSearchCV** y **RandomizedSearchCV** siguiendo la lógica de la Actividad 2.  
La métrica principal será **PR-AUC** (Average Precision), porque la clase churn suele ser minoritaria y esta métrica refleja mejor el rendimiento sobre esa clase.


In [ ]:
# Grillas / distribuciones
param_grid_linear = {
    "clf__C": [0.1, 1, 10]
}

param_grid_rbf = {
    "clf__C": [0.1, 1, 10],
    "clf__gamma": [0.001, 0.01, 0.1, 1]
}

# ==============
# SVM LINEAL
# ==============
grid_linear = GridSearchCV(
    estimator=pipe_svm_linear,
    param_grid=param_grid_linear,
    cv=cv5,
    scoring="average_precision",
    n_jobs=-1,
    refit=True
)

t0 = time.perf_counter()
grid_linear.fit(X_train, y_train)
grid_linear_time = time.perf_counter() - t0

rand_linear = RandomizedSearchCV(
    estimator=pipe_svm_linear,
    param_distributions=param_grid_linear,
    n_iter=10,
    cv=cv5,
    scoring="average_precision",
    n_jobs=-1,
    random_state=RANDOM_STATE,
    refit=True
)

t0 = time.perf_counter()
rand_linear.fit(X_train, y_train)
rand_linear_time = time.perf_counter() - t0

print("SVM lineal | Grid time:", round(grid_linear_time, 3), "s | Best PR-AUC:", grid_linear.best_score_)
print("SVM lineal | Rand time:", round(rand_linear_time, 3), "s | Best PR-AUC:", rand_linear.best_score_)

best_linear_search = grid_linear if grid_linear.best_score_ >= rand_linear.best_score_ else rand_linear
best_svm_linear = best_linear_search.best_estimator_
print("Mejor búsqueda (lineal):", type(best_linear_search).__name__)
pprint(best_linear_search.best_params_)

# ==============
# SVM RBF
# ==============
grid_rbf = GridSearchCV(
    estimator=pipe_svm_rbf,
    param_grid=param_grid_rbf,
    cv=cv5,
    scoring="average_precision",
    n_jobs=-1,
    refit=True
)

t0 = time.perf_counter()
grid_rbf.fit(X_train, y_train)
grid_rbf_time = time.perf_counter() - t0

rand_rbf = RandomizedSearchCV(
    estimator=pipe_svm_rbf,
    param_distributions=param_grid_rbf,
    n_iter=12,
    cv=cv5,
    scoring="average_precision",
    n_jobs=-1,
    random_state=RANDOM_STATE,
    refit=True
)

t0 = time.perf_counter()
rand_rbf.fit(X_train, y_train)
rand_rbf_time = time.perf_counter() - t0

print("SVM RBF   | Grid time:", round(grid_rbf_time, 3), "s | Best PR-AUC:", grid_rbf.best_score_)
print("SVM RBF   | Rand time:", round(rand_rbf_time, 3), "s | Best PR-AUC:", rand_rbf.best_score_)

best_rbf_search = grid_rbf if grid_rbf.best_score_ >= rand_rbf.best_score_ else rand_rbf
best_svm_rbf = best_rbf_search.best_estimator_
print("Mejor búsqueda (RBF):", type(best_rbf_search).__name__)
pprint(best_rbf_search.best_params_)


### 2.2 Evaluación final en TEST (SVM lineal y SVM RBF)

Se reportan las métricas solicitadas en el conjunto de test y se grafican las curvas ROC y Precision–Recall.  
Esto permite comparar desempeño realista y también visualizar el compromiso entre detectar churn y evitar falsas alarmas.


In [ ]:
def evaluate_on_test(model, X_train, y_train, X_test, y_test, title="Modelo"):
    t0 = time.perf_counter()
    model.fit(X_train, y_train)
    fit_time = time.perf_counter() - t0

    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1v = f1_score(y_test, y_pred, zero_division=0)
    rocA = roc_auc_score(y_test, y_proba)
    prA = average_precision_score(y_test, y_proba)

    print(f"\n=== {title} (TEST) ===")
    print("Tiempo de entrenamiento (aprox.):", round(fit_time, 3), "s")
    print("Accuracy :", round(acc, 3))
    print("Precision:", round(prec, 3))
    print("Recall   :", round(rec, 3))
    print("F1       :", round(f1v, 3))
    print("AUC-ROC  :", round(rocA, 3))
    print("PR-AUC   :", round(prA, 3))

    print("\nMatriz de confusión:")
    print(confusion_matrix(y_test, y_pred))

    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)

    p, r, _ = precision_recall_curve(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)

    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.title(f"{title} – Curva ROC (TEST)")
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(r, p, label=f"PR (AP={ap:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"{title} – Curva Precision–Recall (TEST)")
    plt.legend()
    plt.show()

    return {
        "fit_time_s": fit_time,
        "accuracy": acc, "precision": prec, "recall": rec, "f1": f1v, "roc_auc": rocA, "pr_auc": prA
    }

metrics_svm_linear = evaluate_on_test(best_svm_linear, X_train, y_train, X_test, y_test, title="SVM lineal (mejor)")
metrics_svm_rbf = evaluate_on_test(best_svm_rbf, X_train, y_train, X_test, y_test, title="SVM RBF (mejor)")


### 2.3 k-fold (curvas promedio)

A modo complementario, también evaluamos los mejores SVM con k-fold estratificado para observar estabilidad de métricas y curvas promedio.


In [ ]:
df_svm_lin_cv, summary_svm_lin, _ = evaluate_model_cv_detailed(
    best_svm_linear, X, y, cv=cv5, plot_curves=True, verbose=True
)

df_svm_rbf_cv, summary_svm_rbf, _ = evaluate_model_cv_detailed(
    best_svm_rbf, X, y, cv=cv5, plot_curves=True, verbose=True
)


### 2.4 Desbalance: class_weight='balanced'

Se re-entrena el mejor SVM incorporando class_weight='balanced'.  
En general, esta opción busca compensar el desbalance dando más “importancia” a la clase minoritaria, lo que suele aumentar recall de churn, aunque puede afectar la precisión.


In [ ]:
# Aplicar class_weight='balanced' a los mejores modelos
best_svm_linear_bal = best_svm_linear.set_params(clf__class_weight="balanced")
best_svm_rbf_bal = best_svm_rbf.set_params(clf__class_weight="balanced")

metrics_svm_linear_bal = evaluate_on_test(best_svm_linear_bal, X_train, y_train, X_test, y_test, title="SVM lineal (balanced)")
metrics_svm_rbf_bal = evaluate_on_test(best_svm_rbf_bal, X_train, y_train, X_test, y_test, title="SVM RBF (balanced)")


## Paso 3. Comparación final y análisis crítico

En esta sección se comparan Naïve Bayes, SVM lineal y SVM RBF en tres dimensiones:
1) **Desempeño** (en especial F1 y PR-AUC)  
2) **Interpretabilidad**  
3) **Costo computacional** (tiempos de búsqueda/entrenamiento)

Además, se discute por qué el escalamiento es tan importante en SVM y cómo la codificación one-hot afecta la dimensionalidad y el tiempo de entrenamiento.


In [ ]:
# Tabla comparativa (TEST) – resumen de métricas y tiempos aproximados

rows = []

# NB (tiempo de búsqueda + métricas test)
rows.append({
    "model": "Naïve Bayes (GaussianNB)",
    "search_time_s": nb_search_time,
    "fit_time_s": np.nan,
    "accuracy": accuracy_score(y_test, (y_proba_nb >= 0.5).astype(int)),
    "precision": precision_score(y_test, (y_proba_nb >= 0.5).astype(int), zero_division=0),
    "recall": recall_score(y_test, (y_proba_nb >= 0.5).astype(int), zero_division=0),
    "f1": f1_score(y_test, (y_proba_nb >= 0.5).astype(int), zero_division=0),
    "roc_auc": roc_auc_score(y_test, y_proba_nb),
    "pr_auc": average_precision_score(y_test, y_proba_nb)
})

# SVMs
for name, m in [
    ("SVM lineal (mejor)", metrics_svm_linear),
    ("SVM RBF (mejor)", metrics_svm_rbf),
    ("SVM lineal (balanced)", metrics_svm_linear_bal),
    ("SVM RBF (balanced)", metrics_svm_rbf_bal),
]:
    rows.append({
        "model": name,
        "search_time_s": np.nan,   # búsqueda ya reportada arriba; aquí dejamos como referencia simple
        **m
    })

df_compare = pd.DataFrame(rows)
display(df_compare)


### 3.1 Interpretabilidad, desempeño y costo computacional (síntesis)

**Naïve Bayes** suele ser el más rápido y simple, pero su supuesto de independencia puede limitar el desempeño si existen variables redundantes o altamente dependientes.  

**SVM lineal** puede ser una buena alternativa cuando la separación entre clases es relativamente simple y valoramos un modelo más “compacto”.  

**SVM RBF** suele capturar relaciones más complejas, pero puede ser más costoso en tiempo, especialmente con muchas variables (por ejemplo, por efecto del one-hot encoding).  

En problemas de churn, normalmente es útil priorizar métricas enfocadas en la clase positiva (churn), como **PR-AUC** y **F1**, y luego ajustar el umbral o el balance de clases según la capacidad del equipo de retención.


### 3.2 ¿Por qué el escalamiento es crítico en SVM?

SVM se apoya en distancias y en la forma geométrica de los datos. Cuando una variable tiene una escala muy grande, puede dominar esas distancias y hacer que el modelo “preste atención” casi exclusivamente a esa variable, aunque no sea la más relevante. El escalamiento evita ese problema, equilibrando el aporte relativo de cada feature y facilitando que el modelo encuentre una frontera de separación más representativa.


### 3.3 Efecto del one-hot encoding (dimensionalidad y tiempo)

La codificación one-hot transforma variables categóricas en muchas columnas binarias. Esto puede aumentar fuertemente la dimensionalidad, lo que tiene dos efectos principales:  
1) Puede mejorar separabilidad en algunos casos, porque el modelo ve información más detallada.  
2) Puede aumentar el tiempo de entrenamiento, especialmente en métodos como SVM con kernel (RBF), que se vuelven más exigentes cuando crece el número de variables y el tamaño de la matriz.


## Conclusiones

- Se implementó Naïve Bayes con selección de hiperparámetros y evaluación robusta, destacando su rapidez y sus limitaciones cuando hay dependencia entre variables.  
- Se entrenaron y optimizaron SVM lineal y SVM RBF, comparando Grid Search y Random Search en tiempo y desempeño.  
- Se observó que el escalamiento es un requisito práctico para SVM y que el one-hot encoding puede aumentar significativamente la dimensionalidad y el costo computacional.  
- En un escenario de retención, la elección final del modelo debe considerar el equilibrio entre detectar churn (recall) y no saturar al equipo con falsos positivos (precision), apoyándose en PR-AUC, F1 y las curvas Precision–Recall.
